In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
print("""
This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      """)


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [4]:
%matplotlib inline
fig_width_pt = 525  # Get this from LaTeX using \showthe\columnwidth
fig_width_pt = 618  # Get this from LaTeX using \showthe\columnwidth
fig_width_pt = 1024  # Get this from LaTeX using \showthe\columnwidth
ppi = 72.27 # (constant) definition of the ppi = points per inch
inches_per_pt = 1.0/ppi  # Convert pt to inches
#inches_per_cm = 1./2.54
figwidth = fig_width_pt*inches_per_pt  # width in inches
phi = (np.sqrt(5) + 1. ) /2 # golden ratio is good for your eyes
dpi_export = 600

In [5]:
from main import init
args = init()

Using filename= ../data/2019-03-19


In [6]:
print(args)

{'w': 28, 'minibatch_size': 100, 'train_batch_size': 50000, 'test_batch_size': 5000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 30, 'offset_max': 35, 'noise': 0.7, 'contrast': 0.8, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'N_phase': 2, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.5, 'dim1': 1000, 'dim2': 1000, 'lr': 0.0001, 'do_adam': True, 'bn1_bn_momentum': 0.0, 'bn2_bn_momentum': 0.0, 'momentum': 0.1, 'epochs': 50, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-19', 'seed': 2019, 'N_cv': 2}


In [7]:
%%writefile train.py
import torch
import easydict

from main import init
args = init()#'debug')

from retina import Display, Retina
from where import Where, WhereNet
from what import WhatNet
where = Where(args)
filename_train = args.filename + '_train.pt'
where.train(filename_train)


Overwriting train.py


In [8]:
%ls -ltrh /tmp/*pt

-rw-r--r--  1 laurentperrinet  wheel   352M Mar 19 14:34 /tmp/dataset_test__0.2_0.3_0.7_0.8_128_30_35_5000.pt
-rw-r--r--  1 laurentperrinet  wheel   3.4G Mar 19 15:29 /tmp/dataset_train__0.2_0.3_0.7_0.8_30_35_6_16_10_2_1.41_128_50000.pt


In [9]:
%ls -ltrh ../data/_tmp_scanning/

total 280
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:28 sf_0__0_141.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:29 sf_0__0_100.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:29 sf_0__0_119.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:30 sf_0__0_238.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:31 sf_0__0_283.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:32 sf_0__0_168.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:33 sf_0__0_200.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:36 bias_deconv__False.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:39 bias_deconv__True.npy
-rw-r--r--  1 laurentperrinet  staff   152B Mar 19 00:24 sf_0__0_336.npy
-rw-r--r--  1 laurentperrinet  staff   152B Mar 19 01:35 sf_0__0_400.npy
-rw-r--r--  1 laurentperrinet  staff   152B Mar 19 01:36 B_sf__0_150.npy
-rw-r--r--  1 laurentperrinet  staff   152B Mar 19 06:27 B_sf__0_178.npy
-rw-r--r--  1 laurentperrine

In [10]:
np.load('../data/_tmp_scanning/bn1_bn_momentum__0_000.npy')

array([1.00616000e-01, 1.00418000e-01, 6.23789118e+02])

## Version used

In [11]:
%load_ext watermark

In [12]:
%watermark -i -h -m -v -p numpy,matplotlib,torch  -r -g -b

2019-03-19T15:30:59+01:00

CPython 3.7.2
IPython 7.3.0

numpy 1.16.2
matplotlib 3.0.3
torch 1.0.1.post2

compiler   : Clang 10.0.0 (clang-1000.11.45.5)
system     : Darwin
release    : 18.2.0
machine    : x86_64
processor  : i386
CPU cores  : 36
interpreter: 64bit
host name  : fortytwo
Git hash   : fc44687aa0d37b5bf294870f27fcf86d17d74914
Git repo   : https://github.com/laurentperrinet/WhereIsMyMNIST
Git branch : master
